In [7]:
from retnet.modeling_retnet import RetNetForCausalLM
from transformers import AutoTokenizer
import torch

In [3]:
model = RetNetForCausalLM.from_pretrained("Spiral-AI/Spiral-RetNet-3b-base")

Loading checkpoint shards: 100%|██████████| 3/3 [00:01<00:00,  2.35it/s]


In [4]:
tokenizer = AutoTokenizer.from_pretrained("Spiral-AI/Spiral-RetNet-3b-base")

In [39]:
def precompute_kv(model, tokenizer, text: str):
    inputs = tokenizer(text, return_tensors="pt").to(model.device)
    with torch.inference_mode():
        outputs = model(
            **inputs, return_dict=True, use_cache=True, forward_impl="parallel"
        )
    return outputs.past_key_values

In [40]:
kv = precompute_kv("最近、秋葉原周辺")

({'prev_key_value': tensor([[[[-2.4094e-02, -6.5970e-02,  2.4424e-02,  ...,  1.3914e-01,
             -1.9623e-02,  1.0824e-02],
            [-1.1119e-01,  3.4180e-01, -3.0129e-02,  ..., -1.8540e-01,
             -1.5277e-02, -1.9835e-01],
            [ 2.8623e-01, -7.1254e-02, -1.1562e-01,  ...,  3.7701e-01,
             -2.9988e-01,  9.1155e-03],
            ...,
            [ 2.1649e-02,  1.1311e-01,  3.5387e-02,  ...,  3.5730e-01,
             -2.3993e-01, -1.0797e-01],
            [-3.7912e-01, -9.7637e-02,  4.1929e-01,  ..., -1.7666e-01,
              3.1505e-01,  1.4701e-01],
            [-7.2384e-02,  9.3733e-02,  8.4913e-02,  ...,  1.6907e-01,
             -9.1203e-02, -7.0103e-02]],
  
           [[ 1.2964e-01,  1.3035e-01,  1.2613e-01,  ..., -1.4944e-01,
              2.3666e-02, -5.0148e-03],
            [-1.9868e-02, -1.4089e-01, -6.4521e-03,  ...,  6.5458e-02,
             -2.1226e-03, -5.1403e-03],
            [ 3.7325e-01,  3.4657e-01,  1.0133e-01,  ...,  2.5007e-01,
  